In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
import re

In [2]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df = pd.read_csv("chat_dataset.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\subas\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text
df['cleaned_text'] = df['message'].apply(clean_text)

In [4]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['sentiment'])

In [14]:
max_words =10000
max_len = 50
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['cleaned_text'])

X = tokenizer.texts_to_sequences(df['cleaned_text'])
X = pad_sequences(X, maxlen=max_len)

y = df['label'].values

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

C:\Users\subas\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [17]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 24s 598ms/step - accuracy: 0.4030 - loss: 1.0851 - val_accuracy: 0.4615 - val_loss: 1.0586
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 145ms/step - accuracy: 0.4290 - loss: 1.0721 - val_accuracy: 0.4615 - val_loss: 1.0543
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.4468 - loss: 1.0683 - val_accuracy: 0.4615 - val_loss: 1.0471
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.4298 - loss: 1.0617 - val_accuracy: 0.4701 - val_loss: 1.0143
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.5006 - loss: 0.9918 - val_accuracy: 0.6325 - val_loss: 0.9780


In [18]:
def predict_sentiment(text):
    text = clean_text(text)
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=max_len)
    prediction = model.predict(padded_seq)
    sentiment = label_encoder.inverse_transform([np.argmax(prediction)])
    return sentiment[0]

In [20]:
print(predict_sentiment("This is a great product"))
print(predict_sentiment("Im undecided about this"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
neutral
